In [1]:
import pandas as pd
import numpy as np
import re
import toolz.itertoolz as itz
import tqdm

pd.set_option('display.max_columns', 500)

In [2]:
fpath = r"C:\Users\student\Downloads\MetaMorpheusVignette\MetaMorpheusVignette\2024-07-01-12-34-36\Task1-SearchTask\AllPeptides.psmtsv"
df = pd.read_csv(fpath, sep='\t')

In [3]:
df.head(3)

,File Name,Scan Number,Scan Retention Time,Num Experimental Peaks,Total Ion Current,Precursor Scan Number,Precursor Charge,Precursor MZ,Precursor Mass,Score,Delta Score,Notch,Base Sequence,Full Sequence,Essential Sequence,Ambiguity Level,"PSM Count (unambiguous, <0.01 q-value)",Mods,Mods Chemical Formulas,Mods Combined Chemical Formula,Num Variable Mods,Missed Cleavages,Peptide Monoisotopic Mass,Mass Diff (Da),Mass Diff (ppm),Protein Accession,Protein Name,Gene Name,Organism Name,Identified Sequence Variations,Splice Sites,Contaminant,Decoy,Peptide Description,Start and End Residues In Protein,Previous Amino Acid,Next Amino Acid,Theoreticals Searched,Decoy/Contaminant/Target,Matched Ion Series,Matched Ion Mass-To-Charge Ratios,Matched Ion Mass Diff (Da),Matched Ion Mass Diff (Ppm),Matched Ion Intensities,Matched Ion Counts,Normalized Spectral Angle,Localized Scores,Improvement Possible,Cumulative Target,Cumulative Decoy,QValue,Cumulative Target Notch,Cumulative Decoy Notch,QValue Notch,PEP,PEP_QValue
0,04-30-13_CAST_Frac5_4uL,15332,108.77263,196.0,2.423887e+06,15323,3.0,1029.18256,3084.52586,29.175,24.130,0,LQTDQAEASKPTWGTQGDFSTTHALPAVK,LQTDQAEASKPTWGTQGDFSTTHALPAVK,LQTDQAEASKPTWGTQGDFSTTHALPAVK,1,6,NaN,NaN,NaN,0,1,3084.51524,0.01061,3.44,Q80TJ1,Calcium-dependent secretion activator 1,"primary:Cadps, synonym:Caps, synonym:Caps1, sy...",Mus musculus,NaN,NaN,N,N,full,[429 to 457],K,V,,T,"[y1+1, y2+1, y3+1, y4+1, y5+1, y6+1, y7+1, y8+...","[y1+1:147.11223, y2+1:246.18114, y3+1:317.2187...","[y1+1:-0.00058, y2+1:-0.00008, y3+1:0.00045, y...","[y1+1:-3.94, y2+1:-0.33, y3+1:1.42, y4+1:-3.56...","[y1+1:12079, y2+1:8293, y3+1:3001, y4+1:75017,...",29,-1.0,,,1,0,0.0,1,0,0.0,1.072884e-06,0.0
1,04-30-13_CAST_Frac5_4uL,19461,130.57716,200.0,2.517585e+07,19459,3.0,822.08228,2463.22500,28.333,23.284,0,LAELEEFINGPNNAHIQQVGDR,LAELEEFINGPNNAHIQQVGDR,LAELEEFINGPNNAHIQQVGDR,1,4,NaN,NaN,NaN,0,0,2463.21425,0.01075,4.36,Q68FD5,Clathrin heavy chain 1,primary:Cltc,Mus musculus,NaN,NaN,N,N,full,[1183 to 1204],R,C,,T,"[y1+1, y2+1, y3+1, y4+1, y5+1, y6+1, y7+1, y8+...","[y1+1:175.11821, y2+1:290.14648, y3+1:347.1658...","[y1+1:-0.00074, y2+1:0.00059, y3+1:-0.00153, y...","[y1+1:-4.26, y2+1:2.04, y3+1:-4.41, y4+1:-7.12...","[y1+1:118202, y2+1:23759, y3+1:147420, y4+1:81...",28,-1.0,,,2,0,0.0,2,0,0.0,1.788139e-07,0.0
2,04-30-13_CAST_Frac5_4uL,16490,114.88310,200.0,1.948514e+07,16489,2.0,883.00028,1763.98601,28.332,23.332,0,KGVNLPGAAVDLPAVSEK,KGVNLPGAAVDLPAVSEK,KGVNLPGAAVDLPAVSEK,1,6,NaN,NaN,NaN,0,1,1763.97815,0.00786,4.46,P52480,Pyruvate kinase PKM,"primary:Pkm, synonym:Pk3, synonym:Pkm2, synony...",Mus musculus,NaN,NaN,N,N,full,[207 to 224],K,D,,T,"[b1+1, b2+1, b3+1, b4+1, b5+1, b6+1, b7+1, b8+...","[b1+1:129.10188, b2+1:186.12483, b3+1:285.1919...","[b1+1:-0.00036, b2+1:0.00113, b3+1:-0.00016, b...","[b1+1:-2.78, b2+1:6.10, b3+1:-0.57, b4+1:-3.06...","[b1+1:289254, b2+1:54459, b3+1:231008, b4+1:10...",28,-1.0,,,3,0,0.0,3,0,0.0,6.556511e-07,0.0


In [4]:
df.columns, df.size

(Index(['File Name', 'Scan Number', 'Scan Retention Time',
        'Num Experimental Peaks', 'Total Ion Current', 'Precursor Scan Number',
        'Precursor Charge', 'Precursor MZ', 'Precursor Mass', 'Score',
        'Delta Score', 'Notch', 'Base Sequence', 'Full Sequence',
        'Essential Sequence', 'Ambiguity Level',
        'PSM Count (unambiguous, <0.01 q-value)', 'Mods',
        'Mods Chemical Formulas', 'Mods Combined Chemical Formula',
        'Num Variable Mods', 'Missed Cleavages', 'Peptide Monoisotopic Mass',
        'Mass Diff (Da)', 'Mass Diff (ppm)', 'Protein Accession',
        'Protein Name', 'Gene Name', 'Organism Name',
        'Identified Sequence Variations', 'Splice Sites', 'Contaminant',
        'Decoy', 'Peptide Description', 'Start and End Residues In Protein',
        'Previous Amino Acid', 'Next Amino Acid', 'Theoreticals Searched',
        'Decoy/Contaminant/Target', 'Matched Ion Series',
        'Matched Ion Mass-To-Charge Ratios', 'Matched Ion Mass Diff 

In [5]:
df['Mods'].value_counts()

Mods
Carbamidomethyl on C                                             1728
Oxidation on M                                                   1194
Carbamidomethyl on C Carbamidomethyl on C                         320
Carbamidomethyl on C Oxidation on M                               209
Phosphoserine on S                                                128
                                                                 ... 
N6-methyllysine on K                                                1
Carbamidomethyl on C Carbamidomethyl on C N-acetylserine on S       1
N-acetylmethionine on M Phosphothreonine on T                       1
5-glutamyl glycerylphosphorylethanolamine on E                      1
Carbamidomethyl on C N6-acetyllysine on K Phosphoserine on S        1
Name: count, Length: 98, dtype: int64

In [6]:
def get_offset(startend): return int(re.search(r'\d+', startend).group())

def get_mod_locs(seq, offset=0):
    #print(seq)
    parts = re.split("\[|\]", str(seq))
    #print(parts)
    mods = []
    try: 
        if '' == parts[0]:
            #print('YES\n')
            mods+=[[parts[2][0]+str(offset), parts[1]]]
            parts=parts[2:]
        #print('CONT')
        for l1, l2 in itz.partition(2, parts):
            mods+=[[l1[-1] + str(offset+len(l1)-1), l2]]
            offset+=len(l1)
    except:
        print(seq, parts)
        

    if mods: return mods
    else: return [[None, None]]


row = 6983
offset = get_offset(df.at[row, 'Start and End Residues In Protein'])
list(get_mod_locs(df.at[row, 'Full Sequence'], offset=offset)), offset

([['M204', 'Common Variable:Oxidation on M'],
  ['H216', 'UniProt:(3S)-3-hydroxyhistidine on H']],
 201)

In [7]:
if 'localized_mods' not in df.columns:
    mods = [get_mod_locs(i, offset=get_offset(j)) for i, j in df[['Full Sequence', 'Start and End Residues In Protein']].values]
    df.insert(17, 'localized_mods', mods)
#df

In [8]:
df2 = df.explode('localized_mods')
#df2.insert(18, 'lm_counts', df.groupby(['Protein Accession'])['localized_mods'].transform('count'))
#df.sort_values(['lm_counts', 'Protein Accession'])

df2[['Modified Residue', 'Specific Modification']] = pd.DataFrame(df2.localized_mods.tolist(), index=df2.index)

,File Name,Scan Number,Scan Retention Time,Num Experimental Peaks,Total Ion Current,Precursor Scan Number,Precursor Charge,Precursor MZ,Precursor Mass,Score,Delta Score,Notch,Base Sequence,Full Sequence,Essential Sequence,Ambiguity Level,"PSM Count (unambiguous, <0.01 q-value)",localized_mods,Mods,Mods Chemical Formulas,Mods Combined Chemical Formula,Num Variable Mods,Missed Cleavages,Peptide Monoisotopic Mass,Mass Diff (Da),Mass Diff (ppm),Protein Accession,Protein Name,Gene Name,Organism Name,Identified Sequence Variations,Splice Sites,Contaminant,Decoy,Peptide Description,Start and End Residues In Protein,Previous Amino Acid,Next Amino Acid,Theoreticals Searched,Decoy/Contaminant/Target,Matched Ion Series,Matched Ion Mass-To-Charge Ratios,Matched Ion Mass Diff (Da),Matched Ion Mass Diff (Ppm),Matched Ion Intensities,Matched Ion Counts,Normalized Spectral Angle,Localized Scores,Improvement Possible,Cumulative Target,Cumulative Decoy,QValue,Cumulative Target Notch,Cumulative Decoy Notch,QValue Notch,PEP,PEP_QValue,Modified Residue,Specific Modification
0,04-30-13_CAST_Frac5_4uL,15332,108.77263,196.0,2.423887e+06,15323,3.0,1029.18256,3084.52586,29.175,24.130,0,LQTDQAEASKPTWGTQGDFSTTHALPAVK,LQTDQAEASKPTWGTQGDFSTTHALPAVK,LQTDQAEASKPTWGTQGDFSTTHALPAVK,1,6,"[None, None]",NaN,NaN,NaN,0,1,3084.51524,0.01061,3.44,Q80TJ1,Calcium-dependent secretion activator 1,"primary:Cadps, synonym:Caps, synonym:Caps1, sy...",Mus musculus,NaN,NaN,N,N,full,[429 to 457],K,V,,T,"[y1+1, y2+1, y3+1, y4+1, y5+1, y6+1, y7+1, y8+...","[y1+1:147.11223, y2+1:246.18114, y3+1:317.2187...","[y1+1:-0.00058, y2+1:-0.00008, y3+1:0.00045, y...","[y1+1:-3.94, y2+1:-0.33, y3+1:1.42, y4+1:-3.56...","[y1+1:12079, y2+1:8293, y3+1:3001, y4+1:75017,...",29,-1.0,,,1,0,0.000000,1,0,0.000000,1.072884e-06,0.000000,None,None
1,04-30-13_CAST_Frac5_4uL,19461,130.57716,200.0,2.517585e+07,19459,3.0,822.08228,2463.22500,28.333,23.284,0,LAELEEFINGPNNAHIQQVGDR,LAELEEFINGPNNAHIQQVGDR,LAELEEFINGPNNAHIQQVGDR,1,4,"[None, None]",NaN,NaN,NaN,0,0,2463.21425,0.01075,4.36,Q68FD5,Clathrin heavy chain 1,primary:Cltc,Mus musculus,NaN,NaN,N,N,full,[1183 to 1204],R,C,,T,"[y1+1, y2+1, y3+1, y4+1, y5+1, y6+1, y7+1, y8+...","[y1+1:175.11821, y2+1:290.14648, y3+1:347.1658...","[y1+1:-0.00074, y2+1:0.00059, y3+1:-0.00153, y...","[y1+1:-4.26, y2+1:2.04, y3+1:-4.41, y4+1:-7.12...","[y1+1:118202, y2+1:23759, y3+1:147420, y4+1:81...",28,-1.0,,,2,0,0.000000,2,0,0.000000,1.788139e-07,0.000000,None,None
2,04-30-13_CAST_Frac5_4uL,16490,114.88310,200.0,1.948514e+07,16489,2.0,883.00028,1763.98601,28.332,23.332,0,KGVNLPGAAVDLPAVSEK,KGVNLPGAAVDLPAVSEK,KGVNLPGAAVDLPAVSEK,1,6,"[None, None]",NaN,NaN,NaN,0,1,1763.97815,0.00786,4.46,P52480,Pyruvate kinase PKM,"primary:Pkm, synonym:Pk3, synonym:Pkm2, synony...",Mus musculus,NaN,NaN,N,N,full,[207 to 224],K,D,,T,"[b1+1, b2+1, b3+1, b4+1, b5+1, b6+1, b7+1, b8+...","[b1+1:129.10188, b2+1:186.12483, b3+1:285.1919...","[b1+1:-0.00036, b2+1:0.00113, b3+1:-0.00016, b...","[b1+1:-2.78, b2+1:6.10, b3+1:-0.57, b4+1:-3.06...","[b1+1:289254, b2+1:54459, b3+1:231008, b4+1:10...",28,-1.0,,,3,0,0.000000,3,0,0.000000,6.556511e-07,0.000000,None,None
3,04-30-13_CAST_Frac5_4uL,14330,103.51459,197.0,1.265034e+06,14322,2.0,1142.07582,2282.13710,27.395,22.395,0,SGYQQAASEHGLVVIAPDTSPR,SGYQQAASEHGLVVIAPDTSPR,SGYQQAASEHGLVVIAPDTSPR,1,4,"[None, None]",NaN,NaN,NaN,0,0,2282.12912,0.00797,3.49,Q9R0P3,S-formylglutathione hydrolase,"primary:Esd, synonym:Es10, synonym:Sid478",Mus musculus,NaN,NaN,N,N,full,[65 to 86],K,G,,T,"[y1+1, y2+1, y3+1, y4+1, y5+1, y6+1, y7+1, y8+...","[y1+1:175.11839, y2+1:272.17108, y3+1:359.2030...","[y1+1:-0.00056, y2+1:-0.00063, y3+1:-0.00065, ...","[y1+1:-3.21, y2+1:-2.34, y3+1:-1.81, y4+1:-3.3...","[y1+1:4817, y2+1:13242, y3+1:5289, y4+1:85014,...",27,-1.0,,,4,0,0.000000,4,0,0.000000,8.940697e-07,0.000000,None,None
4,04-30-13_CAST_Frac5_4uL,25565,163.87398,200.0,1.438836e+07,25564,4.0,854.95069,3415.77366,27.371,22.349,0,REVEDPQVAQLELGGGPGAGDLQTLALEVAQQK,REVEDPQVAQLELGGGPGA

In [9]:
#df2.loc[(df2.localized_mods!=None) & (df2.lm_counts>2)]

In [10]:
quants = df2['lm_counts']*df2['PSM Count (unambiguous, <0.01 q-value)']
df2.insert(19, 'lm_quants', quants)



KeyError: 'lm_counts'

In [ ]:
df2.insert(20, 'lm_sum', df2.groupby('Protein Accession')['lm_quants'].transform('sum'))

fracs = df2['lm_quants']/df2['lm_sum']
df2.insert(21, 'lm_fracs', fracs)

In [ ]:
df2.loc[df2['Protein Accession']=='Q80TJ1']